In [ ]:
#Importing the required libraries
import pandas as pd
from tqdm import tqdm
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle, islice
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
import scipy.sparse as sp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!unzip /content/Modified_SQL_Dataset.csv.zip

Archive:  /content/Modified_SQL_Dataset.csv.zip
  inflating: Modified_SQL_Dataset.csv  


In [ ]:
sql_df=pd.read_csv('/content/Modified_SQL_Dataset.csv')
sql_df.head()

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


#Train-test split

In [ ]:
def train_test_split_fn(dataset):
    y=dataset['Label']
    X=dataset.drop('Label',axis = 1)
    X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,test_size=0.25,random_state=40)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test= train_test_split_fn(sql_df)
print('Train data shape :',X_train.shape)
print('Test data shape:',X_test.shape)
X_train.head(3)

Train data shape : (23189, 1)
Test data shape: (7730, 1)


,Query
20410,SELECT AVG ( theory ) FROM none
920,-5021'+ ( select yadq where 4285 = 4285 or...
9095,select case when 8237 = 9948 then 1 else nul...


#Preprocess the data

In [ ]:
def preprocessing(data):
    data['Query'] = data['Query'].str.lower()
    data.drop_duplicates(subset=['Query'],keep=False,inplace=True)
    return data

In [ ]:
X_train = preprocessing(X_train)
X_test = preprocessing(X_test)
print('Deduplicated Train data shape :',X_train.shape)
print('Deduplicated Test data shape:',X_test.shape)
X_train.head(3)

Deduplicated Train data shape : (23162, 1)
Deduplicated Test data shape: (7728, 1)


,Query
20410,select avg ( theory ) from none
920,-5021'+ ( select yadq where 4285 = 4285 or...
9095,select case when 8237 = 9948 then 1 else nul...


#Featurizing the data

In [ ]:
def number_of_single_quotations(stringtxt):
    return len(re.findall("\'", stringtxt))
def number_of_double_quotations(stringtxt):
    return len(re.findall("\"", stringtxt))
def number_of_commas(stringtxt):
    return len(re.findall("\,", stringtxt))
def number_of_punctuations(stringtxt):
    return len(re.findall("[!\"#$%&\'()*+,-.\/:;<=>?@[\\]^_`{|}~]", stringtxt))
def number_of_singlelinecomment(stringtxt):
    return len(re.findall(r'--.*', stringtxt))
def number_of_multilinecomment(stringtxt):
    return len(re.findall(r'\/\*', stringtxt))
def number_of_whitespace(stringtxt):
    return len(re.findall(r'\s+', stringtxt))
def number_of_percentages(stringtxt):
    return len(re.findall(r'%', stringtxt))
def number_of_logicalop(stringtxt):
    return len(re.findall(r'\snot\s|\sand\s|\sor\s|\sxor\s|&&|\|\||!', stringtxt))
def number_of_op(stringtxt):
    return len(re.findall(r'\*|\+|\-|\>|\<|\/|\/', stringtxt)) # +-*/<>
def number_of_hexvalues(stringtxt):
    return len(re.findall(r'0[xX][0-9a-fA-F]+', stringtxt)) 
def number_of_alphabets(stringtxt):
    return len(re.findall(r'[a-z]', stringtxt))   
def number_of_digits(stringtxt):
    return len(re.findall(r'[0-9]', stringtxt)) 
def number_of_DBkeywords(stringtxt):
    return len(re.findall(r'\ssysobjects\s|msysobjects\s|version\s', stringtxt))  
def number_of_execcmd(stringtxt):
    return len(re.findall(r'\sexec\s', stringtxt)) 
def number_of_parentheses(stringtxt):
    return len(re.findall(r'\(|\)', stringtxt))
def number_of_squarebrackets(stringtxt):
    return len(re.findall(r'\[|\]', stringtxt))
def number_of_angularbrackets(stringtxt):
    return len(re.findall(r'\<|\>', stringtxt))
def number_of_curlybrackets(stringtxt):
    return len(re.findall(r'\{|\}', stringtxt))
def number_of_atsymbol(stringtxt):
    return len(re.findall(r'\@', stringtxt))
def number_of_hashsymbol(stringtxt):
    return len(re.findall(r'\#', stringtxt))
def number_of_colonsymbol(stringtxt):
    return len(re.findall(r'\:', stringtxt))
def number_of_semicolonsymbol(stringtxt):
    return len(re.findall(r'\;', stringtxt))
def number_of_equalssymbol(stringtxt):
    return len(re.findall(r'\=', stringtxt))
def number_of_dollarsymbol(stringtxt):
    return len(re.findall(r'\$', stringtxt))
def number_of_nulls(stringtxt):
    return len(re.findall(r'null', stringtxt))
def number_of_floor(stringtxt):
    return len(re.findall(r'floor()', stringtxt))
def number_of_rand(stringtxt):
    return len(re.findall(r'rand()', stringtxt))
def cal_keyword(stringtxt,col,key_col,l):
    for i, query in enumerate(stringtxt[col]):
      count = 0
      words = query.split()
      for word in words:
        if word in l:
          count = count+1
      stringtxt[key_col][i] = count
    return stringtxt

In [ ]:
def featurization(data):
    data['no_sngle_quts']   = data['Query'].apply(number_of_single_quotations)
    data['no_dble_quts']    = data['Query'].apply(number_of_double_quotations)
    data['no_commas']       = data['Query'].apply(number_of_commas)
    data['no_punctn']       = data['Query'].apply(number_of_punctuations)
    data['no_sgle_cmnt']    = data['Query'].apply(number_of_singlelinecomment)
    data['no_mlt_cmnt']     = data['Query'].apply(number_of_multilinecomment)
    data['no_whte_spce']    = data['Query'].apply(number_of_whitespace)
    data['no_prctge']       = data['Query'].apply(number_of_percentages)
    data['no_log_oprtr']    = data['Query'].apply(number_of_logicalop)
    data['no_oprtr']        = data['Query'].apply(number_of_op)
    data['no_hexdcml_valus']= data['Query'].apply(number_of_hexvalues)
    data['no_alphabet']     = data['Query'].apply(number_of_alphabets)
    data['no_digits']       = data['Query'].apply(number_of_digits)
    data['no_db_keywords']  = data['Query'].apply(number_of_DBkeywords)
    data['no_execcmd']      = data['Query'].apply(number_of_execcmd)
    data['no_parentheses']  = data['Query'].apply(number_of_parentheses)
    data['no_squarebracket']= data['Query'].apply(number_of_squarebrackets)
    data['no_curlybrackets']= data['Query'].apply(number_of_curlybrackets)
    data['no_anglebracket'] = data['Query'].apply(number_of_angularbrackets)
    data['no_atsymbol']     = data['Query'].apply(number_of_atsymbol)
    data['no_hash']         = data['Query'].apply(number_of_hashsymbol)
    data['no_colon']        = data['Query'].apply(number_of_colonsymbol)
    data['no_semicolon']    = data['Query'].apply(number_of_semicolonsymbol)
    data['no_equals']       = data['Query'].apply(number_of_equalssymbol)
    data['no_dollarsymbol'] = data['Query'].apply(number_of_dollarsymbol)
    data['no_nulls']        = data['Query'].apply(number_of_nulls)
    data['no_floor']        = data['Query'].apply(number_of_floor)
    data['no_rand']         = data['Query'].apply(number_of_rand)
    data['no_keyword'] = 0
    data = cal_keyword(data, 'Query', 'no_keyword', ["select", "update", "insert", "create", "drop", "alter", "truncate", 
                                                    "rename", "exists", "order", "group", "sleep","count","where"])
    return data

In [ ]:
X_train = featurization(X_train)
X_test = featurization(X_test)
print('Featurized Train data shape :',X_train.shape)
print('Featurized Test data shape:',X_test.shape)
X_train.head(3)

Featurized Train data shape : (23162, 30)
Featurized Test data shape: (7728, 30)


,Query,no_sngle_quts,no_dble_quts,no_commas,no_punctn,no_sgle_cmnt,no_mlt_cmnt,no_whte_spce,no_prctge,no_log_oprtr,...,no_atsymbol,no_hash,no_colon,no_semicolon,no_equals,no_dollarsymbol,no_nulls,no_floor,no_rand,no_keyword
20410,select avg ( theory ) from none,0,0,0,2,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,2
920,-5021'+ ( select yadq where 4285 = 4285 or...,1,0,0,6,0,0,10,0,0,...,0,1,0,0,1,0,0,0,0,0
9095,select case when 8237 = 9948 then 1 else nul...,0,0,0,3,1,0,10,0,0,...,0,0,0,0,1,0,1,0,0,3


#Tokenization of data

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=40000)
vectorizer.fit(X_train['Query'])
X_train_tokens = vectorizer.transform(X_train['Query'])
X_test_tokens  = vectorizer.transform(X_test['Query'])

In [ ]:
X_train['Query'].head(1)

20410    select avg ( theory ) from none
Name: Query, dtype: object

In [ ]:
print(X_train_tokens.shape)
print(X_train_tokens[0])

(23162, 40000)
  (0, 30326)	0.499480638189775
  (0, 30320)	0.42783060018354113
  (0, 27503)	0.2436145171396159
  (0, 26424)	0.07932766503800522
  (0, 18635)	0.44269932648221194
  (0, 10799)	0.487947558072798
  (0, 9765)	0.09173664567879625
  (0, 7200)	0.2436145171396159


In [ ]:
X_train= X_train.drop('Query',axis=1)
X_test = X_test.drop('Query',axis=1)

In [ ]:
X_train.head(1)

,no_sngle_quts,no_dble_quts,no_commas,no_punctn,no_sgle_cmnt,no_mlt_cmnt,no_whte_spce,no_prctge,no_log_oprtr,no_oprtr,...,no_atsymbol,no_hash,no_colon,no_semicolon,no_equals,no_dollarsymbol,no_nulls,no_floor,no_rand,no_keyword
20410,0,0,0,2,0,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
out_X_train = hstack((X_train, X_train_tokens)).tocsr()
out_X_test  = hstack((X_test, X_test_tokens)).tocsr()

In [ ]:
print(out_X_train.shape)
print(out_X_test.shape)

(23162, 40029)
(7728, 40029)


#Saving the data

In [ ]:
sp.save_npz("train_features.npz", out_X_train)
sp.save_npz("test_features.npz", out_X_test)
#loaded_train_features = sp.load_npz("train_features.npz")
#loaded_test_features = sp.load_npz("test_features.npz")